# Setting up our environment
Here, we install necessary packages and import required modules.

In [ ]:
# Install OSS Vizier.
!pip install google-vizier

In [ ]:
from concurrent import futures
import grpc
import portpicker

from vizier.service import clients
from vizier.service import pyvizier as vz
from vizier.service import vizier_server
from vizier.service import vizier_service_pb2_grpc

## Setting up the problem statement

Here we setup the problem statement, which contains information about the search space and the metrics to optimize.

In [ ]:
problem = vz.ProblemStatement()
problem.search_space.select_root().add_float_param('x', 0.0, 1.0)
problem.search_space.select_root().add_float_param('y', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='maximize_metric', goal=vz.ObjectiveMetricGoal.MAXIMIZE))


def evaluate(x: float, y: float) -> float:
  return x**2 - y**2

# Setting up the study configuration

The study configuration contains additional information, such as the algorithm to use and level of noise that we think the objective will have.

In [ ]:
study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = vz.Algorithm.RANDOM_SEARCH

# Setting up the server
The server will be run locally here, and will process all client requests.


In [ ]:
port = portpicker.pick_unused_port()
address = f'localhost:{port}'

# Setup server.
server = grpc.server(futures.ThreadPoolExecutor(max_workers=100))

# Setup Vizier Service.
servicer = vizier_server.VizierService()
vizier_service_pb2_grpc.add_VizierServiceServicer_to_server(servicer, server)
server.add_secure_port(address, grpc.local_server_credentials())

# Start the server.
server.start()

# Setting up the client
Starts a client, which will connect to the server.

In [ ]:
clients.environment_variables.service_endpoint = address
study = clients.Study.from_study_config(study_config, owner='owner', study_id='example_study_id')

# Obtaining suggestions
Start requesting suggestions from the server, for evaluating objectives.

In [ ]:
suggestions = study.suggest(count=5)
for suggestion in suggestions:
  x = suggestion.parameters['x']
  y = suggestion.parameters['y']
  print('Suggested Parameters (x,y):', x, y)
  final_measurement = vz.Measurement({'maximize_metric': evaluate(x, y)})
  suggestion.complete(final_measurement)

# Find optimal trial
Find the best objective so far, with corresponding suggestion value. For multiobjective cases, there may be multiple outputs of `optimal_trials()`, all corresponding to a Pareto-optimal curve.

In [ ]:
for optimal_trial in study.optimal_trials():
  optimal_trial = optimal_trial.materialize()
  print("Optimal Trial Suggestion and Objective:", optimal_trial.parameters,
        optimal_trial.final_measurement)